In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = API_KEY

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
import pickle

In [15]:
def ask_no_history(qa, query):
    return qa({"question": query, "chat_history": []})

def ask_with_history_and_append(qa, query, history):
    res = qa({"question": query, "chat_history": history})
    history.append((query, res["answer"]))
    return res

def pretty_print_res(query, res, include_sources=True):
    print(f"[USER]: {query}")
    print(f"[PenDragon]: {res['answer']}\n")
    print("[PenDragon Sources Consulted]:")
    
    for idx, source in enumerate(res["source_documents"]):
        print(f"--- [{idx+1}] {source.page_content}")

In [4]:
embeddings = OpenAIEmbeddings()

In [6]:
db = None
with open("./db_everything_ultra_processed.pkl", "rb") as f:
    db = pickle.load(f)

In [34]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0.7),
    db.as_retriever(search_kwargs={"k": 10}),
    return_source_documents=True
)

In [35]:
chat_history = []

In [28]:
query = "Which CS professors are on leave?"
res = ask_with_history_and_append(qa, query, chat_history)
pretty_print_res(query, res)

[USER]: Which CS professors are on leave?
[PenDragon]:  Samuel McCauley, Aaron M. Williams, and Shikha Singh are on leave.

[PenDragon Sources Consulted]:
--- [1] Samuel McCauley, Assistant Professor of Computer Science; on leave 2022-2023 

--- [2] Aaron M. Williams, Associate Professor of Computer Science; on leave Spring 2023 

--- [3] Shikha Singh, Assistant Professor of Computer Science; on leave 2022-2023 

--- [4] Iris Howley, Assistant Professor of Computer Science 

--- [5] Mark Hopkins, Assistant Professor of Computer Science 

--- [6] Bill K. Jannen, Assistant Professor of Computer Science 

--- [7] Kelly A. Shaw, Professor of Computer Science 

--- [8] Stephen N. Freund, Chair and A. Barton Hepburn Professor of Computer Science 

--- [9] James M. Bern, Assistant Professor of Computer Science 

--- [10] Katie A. Keith, Assistant Professor of Computer Science 



In [30]:
query = "What courses are they teaching in 2024?"
res = ask_with_history_and_append(qa, query, chat_history)
pretty_print_res(query, res)

[USER]: What courses are they teaching in 2024?
[PenDragon]:  Samuel McCauley will be teaching CSCI 256 Section 01 and CSCI 256 Section 02 in Spring 2024. Aaron M. Williams will be teaching CSCI 361 Section 01 and CSCI 361 Section 04 in Spring 2024. Shikha Singh will be teaching CSCI 134 Section 02 and CSCI 134 Section 03 in Spring 2024.

[PenDragon Sources Consulted]:
--- [1] Samuel McCauley, Assistant Professor of Computer Science; on leave 2022-2023 

--- [2] Aaron M. Williams, Associate Professor of Computer Science; on leave Spring 2023 

--- [3] Shikha Singh, Assistant Professor of Computer Science; on leave 2022-2023 

--- [4] CSCI 256 Section 02 year: 2024, semester: Spring, courseID: 010806, sectionType: in-person, peoplesoftNumber: 3204, classType: Lecture, instructors: [{'id': 11194, 'name': 'Samuel McCauley'}], meetings: [{'days': 'MR', 'start': '14:35', 'end': '15:50', 'facility': ''}]

--- [5] CSCI 256 Section 01 year: 2024, semester: Spring, courseID: 010806, sectionType

As an academic advisor at Williams College who is knowledgable various courses and majors, answer the following questions from students in a friendly and concise manner.

In [33]:
query = "As an academic advisor at Williams College who is knowledgable various courses and majors, answer the following questions from students in a friendly and concise manner."
res = ask_with_history_and_append(qa, query, chat_history)
pretty_print_res(query, res)

[USER]: As an academic advisor at Williams College who is knowledgable various courses and majors, answer the following questions from students in a friendly and concise manner.
[PenDragon]:  Hi there! If you're considering taking courses abroad for major/concentration credit, you'll need to have them pre-approved prior to going abroad. You'll need to provide course title and description, and the complete syllabus, including readings and assignments. If you're considering honors work, you'll need to obtain permission from the department before registering in the fall of your senior year. If you're considering designing a plan of study outside of your major, you can discuss your plans in detail with a member of the faculty. If you're looking for introductory courses, you can find more detailed information on the offerings in the department at http://www.cs.williams.edu/. Feel free to contact a member of the department for guidance in selecting a first course.

[PenDragon Sources Consult

In [ ]:
query = ""
res = ask_with_history_and_append(qa, query, chat_history)
pretty_print_res(query, res)